In [ ]:
from etrade_track.z_helper import read_xlsx_codes

#
codes0 = read_xlsx_codes('short.xls')
print(codes0)
codes1 = read_xlsx_codes('shortz.xls')
print(codes1)





In [ ]:
from etrade_track.z_helper import *

code = 'sz002245'

week = 4 * 5

download_rsi_data([code], '60m', 120)
print('download finish')

In [ ]:
code = 'sz002245'

pklfile = f'temp/{code}_60m.pkl'

df = pd.read_pickle(pklfile)

print(df.shape)
# print(df)

In [ ]:
si = -16

print(df.iloc[si, :])

from etrade_track.z_algo_rsi import *

rsi = get_rsi_price_hour(df)

print(rsi)

print(rsi[si])

In [99]:
import datetime
ddate = df['date'].values

print(ddate[-1])
dt = ddate[-1]
# dt = datetime.datetime.utcfromtimestamp(dt.astype('O')/1e9)
# print(dt.day)

def getday(dt):
    dt = datetime.datetime.utcfromtimestamp(dt.astype('O')/1e9)
    # print(dt.day)
    return dt.day
    pass

print(getday(dt))

2022-01-07T15:00:00.000000000
7


In [101]:

def rsi_fuck_df(df, si=-16, ei=0, s0pct=3, s1pct=4):
    # 数据准备
    # df 60m数据
    # rsi
    rsiv = RSI(df['close'], 6)
    # rsi 价格
    rsi = get_rsi_price_hour(df)
    # 日期
    date = df['date'].values

    op = []  # 操作记录
    hold = 0  #  买入状态
    holddate = 0 # 买入日期
    holdp = 0  # 买入价格
    holdhigh = 0  # 最高价格
    sell0p = 0  # 止损价格
    sell1p = 0  # 止盈价格

    # 回测范围
    rng = range(si, ei, 1)
    for ri in rng:
        low = df['low'].values[ri]
        high = df['high'].values[ri]
        close = df['close'].values[ri]
        rsip = round(rsi[ri], 2)

        if hold == 0:
            # empty
            # wait to buy
            if low < rsip and rsiv[ri] <= 20:
                # buy
                hold = 1
                holdp = round(rsip, 2)
                holddate = date[ri]
                holdhigh = round(rsip, 2)
                sell1p = round(rsip * (100 + s1pct) / 100, 2)
                sell0p = round(rsip * (100 - s0pct) / 100, 2)
                op.append([1, rsip, date[ri]])
                print(f'buy:{rsip:.2f} {date[ri]}')
                print('sell0p', sell0p)
                print('sell1p', sell1p)

        elif hold > 0 and getday(date[ri]) != getday(holddate):
            # wait to sell
            if low < sell0p:
                # 股价超过止损线
                # 止损
                if hold > 0:
                    op.append([-1, sell0p, date[ri]])
                    print(f'sell0:{sell0p:.2f} {date[ri]}')
                    hold = -1
            elif high >= sell1p:
                # 股价超过止盈线
                # 止盈
                if hold > 0:
                    op.append([0, sell1p, date[ri]])
                    print(f'sell1:{sell1p:.2f} {date[ri]}')
                    hold = -1
            else:
                # 股价上涨更新高点
                if high > holdhigh * (100 + s1pct / 2) / 100:
                    holdhigh = high
                    print(f'update high:{holdhigh}')
                # 冲高
                if holdhigh > holdp:
                    # 回落价格
                    highp = round(holdhigh * (100 - s0pct / 2) / 100, 2)
                    # 高点回落
                    if close < highp:
                        if hold > 0:
                            if highp > holdp:
                                op.append([0, highp, date[ri]])
                                print(f'sell1:{highp:.2f} {date[ri]}')
                            else:
                                # 原价卖出
                                op.append([-1, holdp, date[ri]])
                                print(f'sell0:{holdp:.2f} {date[ri]}')
                            hold = -1

        if hold == -1:
            print('op finish')
            break
        pass

    pct = 0
    if len(op) == 2:
        if op[0][0] == 1:
            p0 = op[0][1]
            p1 = op[1][1]
            pct = round((p1 - p0) / p0 * 100, 2)

    return op, pct


code = 'sz002245'

pklfile = f'temp/{code}_60m.pkl'

df = pd.read_pickle(pklfile)

ops, pct = rsi_fuck_df(df)

for op in ops:
    print(op)
print(pct)

buy:24.68 2022-01-05T11:30:00.000000000
sell0p 23.94
sell1p 25.67
update high:25.31
sell1:24.93 2022-01-06T15:00:00.000000000
op finish
[1, 24.68, numpy.datetime64('2022-01-05T11:30:00.000000000')]
[0, 24.93, numpy.datetime64('2022-01-06T15:00:00.000000000')]
1.01


In [78]:
def formatop(ops):
    if len(ops) == 0:
        return ',,,,,'
    if len(ops) == 1:
        opstr = ','.join([','.join([str(o) for o in op]) for op in ops])
        return opstr + ',,,'

    opstr = ','.join([','.join([str(o) for o in op]) for op in ops])
    #print(opstr)
    return opstr


print(formatop(ops))

,,,,,
,,,,,


In [103]:
code = read_xlsx_codes('short.xls')

print(code)

with open('backtest.csv', 'w') as f:
    for c in code:
        print(f'{c} start ...')
        # download_rsi_data([c], '60m', 120)
        df = pd.read_pickle(f'temp/{c}_60m.pkl')
        ops, pct = rsi_fuck_df(df)
        csvstr = f'{c},{db_id_to_name(c)},{pct:.2f},{formatop(ops)}\n'
        print(csvstr)
        f.write(csvstr)
        print(f'{c} {code.index(c)} finish {pct}...\n\n')

read csv success
['sz002699', 'sh603297', 'sh600180', 'sz002530', 'sh603348', 'sh600426', 'sh603171', 'sh600348', 'sz000810', 'sh603501', 'sz002508', 'sz300182', 'sh600150', 'sz002245', 'sz002884', 'sh600703', 'sz000503', 'sz002460', 'sz000681', 'sz002135', 'sz300213', 'sz002840', 'sz002733', 'sz002746', 'sz002750']
sz002699 start ...
sz002699,美盛文化,0.00,,,,,,

sz002699 0 finish 0...


sh603297 start ...
buy:108.14 2022-01-05T10:30:00.000000000
sell0p 104.9
sell1p 112.47
update high:111.97
sell1:110.29 2022-01-06T10:30:00.000000000
op finish
sh603297,永新光学,1.99,1,108.14,2022-01-05T10:30:00.000000000,0,110.29,2022-01-06T10:30:00.000000000

sh603297 1 finish 1.99...


sh600180 start ...
sh600180,瑞茂通,0.00,,,,,,

sh600180 2 finish 0...


sz002530 start ...
sz002530,金财互联,0.00,,,,,,

sz002530 3 finish 0...


sh603348 start ...
buy:55.09 2022-01-05T10:30:00.000000000
sell0p 53.44
sell1p 57.29
sell1:57.29 2022-01-06T14:00:00.000000000
op finish
sh603348,文灿股份,3.99,1,55.09,2022-01-05T10:30:00.0000